## EST DEVLAND

This notebook removes older files and leaves behind the most recent files in each folder afterwhich it then lists files paths

In [1]:
import nbimporter
import time
import os
import glob
import pandas as pd

In [ ]:
%run react.ipynb

In [3]:
directory = r'R:\RawData\Elite Star\Devland\RAW FILES RECEIVED'

In [ ]:
def remove_except_recent(directory, days):
    # Get a list of all files in the directory and its subdirectories
    files = []
    for root, dirs, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))

    # Sort the files by modification time in descending order
    files.sort(key=os.path.getmtime, reverse=True)

    # Keep track of the most recent file in each directory
    most_recent_files = {}

    # Keep track of the deleted files
    deleted_files = []

    # Keep track of the kept files
    kept_files = []

    # Calculate the time threshold for deletion
    threshold_time = time.time() - days * 24 * 60 * 60

    # Keep the most recent file in each directory and delete the rest
    for file in files:
        # Get the directory of the file
        directory = os.path.dirname(file)

        # If there is only one file in the directory, skip it
        if len(glob.glob(os.path.join(directory, "*"))) == 1:
            kept_files.append(os.path.basename(file))
            continue

        # If the directory is not in the most_recent_files dictionary, add it
        if directory not in most_recent_files:
            most_recent_files[directory] = file
            kept_files.append(os.path.basename(file))
        else:
            # If the file is older than the most recent file in the directory and older than the threshold, delete it
            if os.path.getmtime(file) < os.path.getmtime(most_recent_files[directory]) and os.path.getmtime(file) < threshold_time:
                os.remove(file)
                deleted_files.append(os.path.basename(file))
            else:
                # Otherwise, update the most recent file in the directory
                os.remove(most_recent_files[directory])
                deleted_files.append(os.path.basename(most_recent_files[directory]))
                most_recent_files[directory] = file
                kept_files.append(os.path.basename(file))

    # Keep the most recent file in each directory
    for directory, most_recent_file in most_recent_files.items():
        files_in_directory = glob.glob(os.path.join(directory, "*"))
        for file in files_in_directory:
            if file != most_recent_file and os.path.getmtime(file) < threshold_time:
                os.remove(file)
                deleted_files.append(os.path.basename(file))
            else:
                kept_files.append(os.path.basename(file))

    # Create a Pandas DataFrame of the deleted files
    df_deleted = pd.DataFrame(deleted_files, columns=["Deleted Files"])
    df_deleted["Deleted Files"] = df_deleted["Deleted Files"].apply(lambda x: os.path.basename(x))
    df_deleted = df_deleted.sort_values(by=["Deleted Files"])

    # Create a Pandas DataFrame of the kept files with only the last two elements in the file path
    df_kept = pd.DataFrame(kept_files, columns=["Kept Files"])
    df_kept["Kept Files"] = df_kept["Kept Files"].apply(lambda x: os.path.join(*os.path.split(x)[-2:]))
    df_kept = df_kept.drop_duplicates()
    
    return df_deleted, df_kept



In [6]:
def list_all_files(directory):
    file_paths = []

    # Walk through the directory
    for root, dirs, filenames in os.walk(directory):
        for filename in filenames:
            # Get the relative path of the file from the directory
            relative_path = os.path.relpath(os.path.join(root, filename), directory)
            
            file_paths.append(relative_path)

    df = pd.DataFrame(file_paths, columns=['File Path'])

    return df

In [ ]:
delete_df, kept_df = delete_all_except_recent(directory)

In [7]:
remaining_files = list_all_files(directory)

In [9]:
remaining_files

,File Path
0,BOKSBURG SNACKS AND DRINKS\SaleAudit_BOKSBURG ...
1,BOKSBURG SNACKS AND DRINKS\SaleAudit_BOKSBURG ...
2,BOKSBURG SNACKS AND DRINKS\SaleAudit_BOKSBURG ...
3,BOKSBURG SNACKS AND DRINKS\SaleAudit_BOKSBURG ...
4,BOKSBURG SNACKS AND DRINKS\SaleAudit_BOKSBURG ...
...,...
151,Newtown Cash & Carry\SaleAudit_Newtown Cash & ...
152,Newtown Cash & Carry\SaleAudit_Newtown Cash & ...
153,Newtown Cash & Carry\SaleAudit_Newtown Cash & ...
154,Newtown Cash & Carry\SaleAudit_Newtown Cash & ...


In [8]:
remaining_files.to_excel("C:\\Users\\tsello01\\Documents\\Data\\devland_files(3).xlsx", index=False)
